# Experiment 001: Baseline Verification

Load the best pre-optimized submission and verify its score.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import shutil
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_VERTICES = list(zip(TX, TY))

print("Tree geometry loaded")
print(f"Number of vertices: {len(TREE_VERTICES)}")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon(TREE_VERTICES)
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def parse_value(val):
    """Parse value with 's' prefix"""
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    """Load trees for configuration N"""
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def compute_bounding_side(trees):
    """Compute bounding box side length"""
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    side = max(all_points.max(axis=0) - all_points.min(axis=0))
    return side

def compute_total_score(df):
    """Compute total score for submission"""
    total = 0
    for n in range(1, 201):
        trees = load_trees_for_n(df, n)
        side = compute_bounding_side(trees)
        total += side**2 / n
    return total

print("Functions defined")

In [ ]:
# Load the best pre-optimized submission
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv'

# Check if file exists
if os.path.exists(preopt_path):
    df = pd.read_csv(preopt_path)
    print(f"Loaded {len(df)} rows from santa-2025.csv")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst 10 rows:")
    print(df.head(10))
else:
    print(f"File not found: {preopt_path}")
    # Try alternative path
    alt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
    if os.path.exists(alt_path):
        df = pd.read_csv(alt_path)
        print(f"Loaded from alternative path: {len(df)} rows")

In [ ]:
# Compute total score
print("Computing total score...")
score = compute_total_score(df)
print(f"\nTotal Score: {score:.6f}")
print(f"Target Score: 68.919154")
print(f"Gap: {score - 68.919154:.6f} ({(score - 68.919154) / 68.919154 * 100:.2f}%)")

In [ ]:
# Analyze per-N scores
print("\nPer-N Analysis (top 20 contributors to score):")
per_n_scores = []
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    side = compute_bounding_side(trees)
    contribution = side**2 / n
    per_n_scores.append({'n': n, 'side': side, 'contribution': contribution, 'efficiency': n / (side**2)})

per_n_df = pd.DataFrame(per_n_scores)
per_n_df = per_n_df.sort_values('contribution', ascending=False)
print(per_n_df.head(20).to_string())

In [ ]:
# Check for overlaps in a few configurations
def has_overlap(trees):
    """Check if any trees overlap"""
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Checking for overlaps in sample configurations...")
for n in [1, 10, 50, 100, 200]:
    trees = load_trees_for_n(df, n)
    overlap = has_overlap(trees)
    print(f"N={n}: {'OVERLAP DETECTED!' if overlap else 'No overlaps'} (side={compute_bounding_side(trees):.4f})")

In [ ]:
# Copy to submission directory
shutil.copy(preopt_path, '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows")

In [ ]:
# Also save to experiment folder
shutil.copy(preopt_path, '/home/code/experiments/001_baseline/submission.csv')
print("Saved to experiment folder")

# Summary
print(f"\n" + "="*50)
print(f"BASELINE SUMMARY")
print(f"="*50)
print(f"Score: {score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {score - 68.919154:.6f}")
print(f"Rows: {len(df)}")
print(f"Submission saved to: /home/submission/submission.csv")